In [1]:
%load_ext gams_magic

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import *
from tensorflow.keras import backend as K

2.12.0


In [4]:
import sys
import os
import inspect

In [5]:
pwd = os.getcwd()
pardir = os.path.abspath(os.path.join(pwd, os.pardir))
print(pardir)

C:\Users\tk3016\OneDrive - Imperial College London\PhD


In [116]:
path = os.path.join(pwd, 'Case1_slle')
#tanh_path = os.path.join(pardir,'tanh_model_ib_noT_miL2')
#relu_path = os.path.join(pardir,'relu_model_ib_noT_miL2')
#rbf_path = os.path.join(pardir,'rbf_model_ib_noT_miL2')

In [117]:
tanh_model = tf.keras.models.load_model(path, compile=False)
tanh_model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 6)                13        
 n)                                                              
                                                                 
 dense_72 (Dense)            (None, 8)                 56        
                                                                 
 dense_73 (Dense)            (None, 1)                 9         
                                                                 
Total params: 78
Trainable params: 65
Non-trainable params: 13
_________________________________________________________________


In [118]:
weights = tanh_model.get_weights()

In [119]:
print(weights)

[array([2.58356   , 2.5948727 , 2.7891212 , 2.8963368 , 0.333398  ,
       0.33402148], dtype=float32), array([0.5847821 , 0.6174661 , 0.8918795 , 1.4750435 , 0.03240524,
       0.03249015], dtype=float32), 12898, array([[ 3.20951056e+00,  1.91556466e+00,  9.72575247e-01,
        -1.24344718e+00, -4.00893831e+00, -2.43695521e+00,
        -6.37732923e-01, -3.27176285e+00],
       [ 1.72401547e+00,  1.70089066e-01,  1.43907535e+00,
        -2.14789128e+00, -1.64370537e-01, -7.10331351e-02,
        -1.61523187e+00, -1.19967237e-01],
       [-7.47406423e-01, -2.43979621e+00, -1.40649784e+00,
         3.26730400e-01,  6.09438753e+00,  1.94798338e+00,
         1.06608808e+00,  3.86388135e+00],
       [ 8.52365434e-01,  1.13742220e+00,  1.27677226e+00,
        -1.22907722e+00, -7.84583211e-01, -2.66059637e-01,
        -1.94601476e+00, -8.12177002e-01],
       [ 1.97248185e+00,  1.22557771e+00,  1.10452306e+00,
        -6.65664002e-02, -1.12323254e-01,  3.56652141e-01,
         2.64967942e+00,

In [120]:
mean = weights[0]
sd = weights[1]**0.5

In [121]:
print(mean)
print(sd)

[2.58356    2.5948727  2.7891212  2.8963368  0.333398   0.33402148]
[0.7647105  0.7857901  0.9443937  1.2145137  0.18001455 0.18025024]


In [122]:
input_gain = 1/sd
input_offset = mean
print(input_gain)
print(input_offset)

[1.3076844  1.2726045  1.0588804  0.82337487 5.5551066  5.547843  ]
[2.58356    2.5948727  2.7891212  2.8963368  0.333398   0.33402148]


In [123]:
num_inputs = len(input_gain)
print(num_inputs)

6


In [124]:
%%gams
SETS
f      'inputs to neural network' /1*6/;

In [125]:
%%gams
SETS
hl1    'number of hidden layers' /1*8/;

In [126]:
%%capture
%run DataTransform.ipynb

In [127]:
%gams_pull f hl1

In [128]:
%%gams
parameter input_offset(f), input_gain(f), bias1(hl1), wt2(hl1), wt1(f<,hl1<);

In [129]:
print(input_offset)
%gams_push input_offset
%gams display input_offset;
%gams_lst -e
%gams_cleanup -k

[2.58356    2.5948727  2.7891212  2.8963368  0.333398   0.33402148]
E x e c u t i o n


----    626 PARAMETER input_offset  1-dim Matrix

1 2.584,    2 2.595,    3 2.789,    4 2.896,    5 0.333,    6 0.334





In [130]:
print(input_gain)
%gams_push input_gain
%gams display input_gain;
%gams_lst -e
%gams_cleanup -k

[1.3076844  1.2726045  1.0588804  0.82337487 5.5551066  5.547843  ]
E x e c u t i o n


----    632 PARAMETER input_gain  1-dim Matrix

1 1.308,    2 1.273,    3 1.059,    4 0.823,    5 5.555,    6 5.548





In [131]:
bias1 = weights[4]
print(bias1.shape)
%gams_push bias1
%gams display bias1;
%gams_lst -e
%gams_cleanup -k

(8,)
E x e c u t i o n


----    638 PARAMETER bias1  1-dim Matrix

1  1.845,    2  1.976,    3 -0.077,    4  0.095,    5 -1.266,    6 -2.191,    7 -1.222,    8 -2.216





In [132]:
%%gams
Scalar bias2;

In [133]:
bias2 = weights[6]
bias2 = float(bias2[0])
print(bias2)
%gams_push bias2
%gams display bias2;
%gams_lst -e
%gams_cleanup -k

7.1846160888671875
E x e c u t i o n


----    648 PARAMETER bias2                =        7.185  




In [134]:
wt2 = weights[5]
print(wt2)
print(wt2.shape)
wt2 = np.reshape(wt2, (8,))
print(wt2.shape)
%gams_push wt2
%gams display wt2;
%gams_lst -e
%gams_cleanup -k

[[ 4.136224 ]
 [ 3.6127055]
 [ 3.9071915]
 [-4.1957693]
 [-7.109919 ]
 [-6.1642976]
 [-6.084468 ]
 [-6.231771 ]]
(8, 1)
(8,)
E x e c u t i o n


----    654 PARAMETER wt2  1-dim Matrix

1  4.136,    2  3.613,    3  3.907,    4 -4.196,    5 -7.110,    6 -6.164,    7 -6.084,    8 -6.232





In [135]:
wt1 = weights[3]
print(wt1.shape)
wt1 = pd.DataFrame(index=f,columns=hl1, data=wt1)
print(wt1)
wt1 = gt_from2dim(wt1)
#print(wt1)
%gams_push wt1
%gams display wt1;
%gams_lst -e
%gams_cleanup -k

(6, 8)
          1         2         3         4         5         6         7  \
1  3.209511  1.915565  0.972575 -1.243447 -4.008938 -2.436955 -0.637733   
2  1.724015  0.170089  1.439075 -2.147891 -0.164371 -0.071033 -1.615232   
3 -0.747406 -2.439796 -1.406498  0.326730  6.094388  1.947983  1.066088   
4  0.852365  1.137422  1.276772 -1.229077 -0.784583 -0.266060 -1.946015   
5  1.972482  1.225578  1.104523 -0.066566 -0.112323  0.356652  2.649679   
6 -2.197256 -0.947228 -3.893650  3.428297  0.003810 -0.177014 -0.245976   

          8  
1 -3.271763  
2 -0.119967  
3  3.863881  
4 -0.812177  
5  0.425975  
6 -0.986781  
E x e c u t i o n


----    661 PARAMETER wt1  

            1           2           3           4           5           6           7           8

1       3.210       1.916       0.973      -1.243      -4.009      -2.437      -0.638      -3.272
2       1.724       0.170       1.439      -2.148      -0.164      -0.071      -1.615      -0.120
3      -0.747      -2.440

In [136]:
%%gams
Free Variables
inp(f)
inter1v(hl1)
inter2v
inter1z(hl1)
inter2z
z
obj
;

Equations
eq_inter1v(hl1)
eq_inter1z(hl1)
eq_inter2v
eq_inter2z
eq_z
eq_obj
;

eq_inter1v(hl1)..  					inter1v(hl1) =e= sum(f, wt1(f,hl1)*inp(f)) + bias1(hl1);
*eq_inter1z(hl1)..  					inter1z(hl1) =e= 1 - (2/(exp(2*inter1v(hl1))+1));
eq_inter1z(hl1)..  					inter1z(hl1) =e= tanh(inter1v(hl1));
*eq_inter1z(hl1)..                   inter1z(hl1) =e= exp(-inter1v(hl1)*inter1v(hl1));
eq_inter2v..     					inter2v =e= sum(hl1, wt2(hl1)*inter1z(hl1)) + bias2;
eq_inter2z..     					inter2z =e= 1/(1 + exp(-inter2v));
eq_z..                              z =e= inter2z;
eq_obj.. obj =e= inter2v;
model NN /all/;

inter2z.lo = 0;
inter2z.up = 1;
inp.lo(f) = -10;
inp.up(f) = 10;
inter1z.lo(hl1) = -1;
inter1z.up(hl1) = 1;

In [137]:
%gams_cleanup -k

In [138]:
%gams solve NN maximising obj using nlp;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),48.627,20,26,NLP,CONOPT,0.062


In [ ]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import LinExpr

In [ ]:
num_layers = 1
num_neurons = 5
num_inputs = 6
LB1_CP = []
UB1_CP = []
x_lb = -50
x_ub = 50
wt1= weights[3]
b1=weights[4]
print(wt1.shape)
print(b1.shape)
for i in range(num_neurons):
    sum_lb = 0
    sum_ub = 0
    for j in range(num_inputs):
    #print(j,i)
        sum_lb += ((x_lb*max(0,wt1[j][i])) + (x_ub*min(0,wt1[j][i])))
        sum_ub += ((x_ub*max(0,wt1[j][i])) + (x_lb*min(0,wt1[j][i])))
    sum_lb += b1[i]
    sum_ub += b1[i]
    LB1_CP.append(sum_lb)
    UB1_CP.append(sum_ub)
print(LB1_CP)
print(UB1_CP)
print(type(LB1_CP[0]))
print(type(UB1_CP[0]))

In [ ]:
for i in range(0,len(UB1_CP)):
    if UB1_CP[i] < 0:
        UB1_CP[i] = 0
print(UB1_CP)

In [ ]:
wt2 = weights[5]
b2 = weights[6]
print(wt2.shape)
print(b2.shape)
print(wt2)

In [ ]:
m = gp.Model("OBBT")
x = m.addVars (num_inputs, lb=-50.0, ub=50.0, vtype=GRB.CONTINUOUS)
inter1z = m.addVars(num_neurons, lb=0, ub=UB1_CP, vtype=GRB.CONTINUOUS)
inter1v = m.addVars(num_neurons, lb=LB1_CP, ub=UB1_CP, vtype=GRB.CONTINUOUS)
y1 = m.addVars(num_neurons, vtype=GRB.BINARY)
inter2v = m.addVar(lb=-10, ub=10, vtype=GRB.CONTINUOUS)
lb = [None]*num_neurons
ub = [None]*num_neurons
linexp = LinExpr()
for i in range(num_neurons):
    for j in range(num_inputs):
        linexp += LinExpr(wt1[j][i],x[j])
    linexp += b1[i]
    m.addConstr(inter1v[i] - linexp == 0)
    m.addConstr(inter1z[i] >= inter1v[i])
    m.addConstr(inter1z[i] <= inter1v[i] - (1 - y1[i])*LB1_CP[i])
    m.addConstr(inter1z[i] <= y1[i]*UB1_CP[i])
linexp1 = LinExpr()
for i in range(num_neurons):
    linexp1 += LinExpr(wt2[i][0], inter1z[i])
linexp1 += b2
m.addConstr(inter2v - linexp1 == 0) 

In [ ]:
LB1_0 = [None]*num_neurons
UB1_0 = [None]*num_neurons
for i in range(num_neurons):
    m.setParam('TimeLimit',120)
    m.setParam('LogToConsole',1)
    m.setObjective(inter1v[i],GRB.MINIMIZE)
    m.optimize()
    if m.getAttr(GRB.Attr.SolCount) != 0:
        if m.getAttr(GRB.Attr.Status) == GRB.OPTIMAL:
            lb[i] = m.getAttr(GRB.Attr.ObjVal)
        else:
            lb[i] = m.getAttr(GRB.Attr.ObjBound)
    if lb[i] > LB1_CP[i]:
        LB1_0[i] = lb[i]
    else:
        LB1_0[i] = LB1_CP[i]
        print('Improved lower bound not found!')           
    m.reset()
    m.setObjective(inter1v[i], GRB.MAXIMIZE)
    m.optimize()
    if m.getAttr(GRB.Attr.SolCount) != 0:
        if m.getAttr(GRB.Attr.Status) == GRB.OPTIMAL:
            ub[i] = m.getAttr(GRB.Attr.ObjVal)
        else:
            ub[i] = m.getAttr(GRB.Attr.ObjBound)
    if ub[i] < UB1_CP[i]:
        UB1_0[i] = ub[i]
    else:
        UB1_0[i] = UB1_CP[i]
        print('Improved upper bound not found!')           
    m.reset()

In [ ]:
m = gp.Model("OBBT")
x = m.addVars (num_inputs, lb=-50.0, ub=50.0, vtype=GRB.CONTINUOUS)
inter1z = m.addVars(num_neurons, lb=0, ub=UB1_0, vtype=GRB.CONTINUOUS)
inter1v = m.addVars(num_neurons, lb=LB1_0, ub=UB1_0, vtype=GRB.CONTINUOUS)
y1 = m.addVars(num_neurons, vtype=GRB.BINARY)
inter2v = m.addVar(lb=-10, ub=10, vtype=GRB.CONTINUOUS)
lb = [None]*num_neurons
ub = [None]*num_neurons
linexp = LinExpr()
for i in range(num_neurons):
    for j in range(num_inputs):
        linexp += LinExpr(wt1[j][i],x[j])
    linexp += b1[i]
    m.addConstr(inter1v[i] - linexp == 0)
    m.addConstr(inter1z[i] >= inter1v[i])
    m.addConstr(inter1z[i] <= inter1v[i] - (1 - y1[i])*LB1_CP[i])
    m.addConstr(inter1z[i] <= y1[i]*UB1_CP[i])
linexp1 = LinExpr()
for i in range(num_neurons):
    linexp1 += LinExpr(wt2[i][0], inter1z[i])
linexp1 += b2
m.addConstr(inter2v - linexp1 == 0) 

In [ ]:
x_lb_0 = -50
x_ub_0 = 50
x_lb_1 = [None]*num_inputs
x_ub_1 = [None]*num_inputs
x_lb = [None]*num_inputs
x_ub = [None]*num_inputs
for i in range(num_inputs):
    m.setParam('TimeLimit',120)
    m.setParam('LogToConsole',0)
    m.setObjective(x[i],GRB.MINIMIZE)
    m.optimize()
    if m.getAttr(GRB.Attr.SolCount) != 0:
        if m.getAttr(GRB.Attr.Status) == GRB.OPTIMAL:
            x_lb[i] = m.getAttr(GRB.Attr.ObjVal)
        else:
            x_lb[i] = m.getAttr(GRB.Attr.ObjBound)
    if x_lb[i] > x_lb_0:
        x_lb_1[i] = x_lb[i]
    else:
        x_lb_1[i] = x_lb_0
        print('Improved lower bound not found!')           
    m.reset()
    m.setObjective(x[i], GRB.MAXIMIZE)
    m.optimize()
    if m.getAttr(GRB.Attr.SolCount) != 0:
        if m.getAttr(GRB.Attr.Status) == GRB.OPTIMAL:
            x_ub[i] = m.getAttr(GRB.Attr.ObjVal)
        else:
            x_ub[i] = m.getAttr(GRB.Attr.ObjBound)
    if x_ub[i] < x_ub_0:
        x_ub_1[i] = x_ub[i]
    else:
        x_ub_1[i] = x_ub_0
        print('Improved upper bound not found!')           
    m.reset()

In [ ]:
print(LB1_0)
print(LB1_CP)

In [ ]:
print(UB1_0)
print(UB1_CP)

In [ ]:
%%gams
parameter bigmlb(hl1), bigmub(hl1);

In [ ]:
bigmlb = np.array(LB1_0)
bigmub = np.array(UB1_0)
print(bigmlb.shape)

In [ ]:
%gams_push bigmlb
%gams display bigmlb;
%gams_lst -e
%gams_cleanup -k

In [ ]:
%gams_push bigmub
%gams display bigmub;
%gams_lst -e
%gams_cleanup -k